In [6]:
#Si importano le librerie principali
from keras.callbacks import TensorBoard
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import cv2
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tqdm import tqdm
import os
from pathlib import Path
import itertools
from sklearn.utils import shuffle
from keras import Sequential
from keras.optimizers import SGD,Adam
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, TensorBoard, ModelCheckpoint
from sklearn.metrics import classification_report,confusion_matrix
import ipywidgets as widgets
import io
from keras.layers import Flatten,Dense,BatchNormalization,Activation,Dropout
from keras.utils import to_categorical
from PIL import Image
from IPython.display import display,clear_output
from warnings import filterwarnings
from tensorflow import keras 
from tensorflow.keras import layers
image_size = 150

In [17]:
#a different model with 4 conv layers (no augmentation)
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization

cnn4 = Sequential()
cnn4.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(image_size,image_size,3)))
cnn4.add(BatchNormalization())

cnn4.add(Conv2D(32, kernel_size=(3, 3), activation='relu'))
cnn4.add(BatchNormalization())
cnn4.add(MaxPooling2D(pool_size=(2, 2)))
cnn4.add(Dropout(0.25))

cnn4.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
cnn4.add(BatchNormalization())
cnn4.add(Dropout(0.25))

cnn4.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
cnn4.add(BatchNormalization())
cnn4.add(MaxPooling2D(pool_size=(2, 2)))
cnn4.add(Dropout(0.25))

cnn4.add(Flatten())

cnn4.add(Dense(512, activation='relu'))
cnn4.add(BatchNormalization())
cnn4.add(Dropout(0.5))

cnn4.add(Dense(128, activation='relu'))
cnn4.add(BatchNormalization())
cnn4.add(Dropout(0.5))
cnn4.add(Dense(4, activation='softmax'))

cnn4.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.SGD(),
              metrics=['accuracy'])

In [8]:
labels = ['glioma_tumor','meningioma_tumor','no_tumor','pituitary_tumor']

In [10]:
#Si iniziano a leggere tutte le immagini e ad aggiungerle tramite una lista
#e si convertono in un numpy array dopo averle rese tutte della stessa grandezza
X_train = []
y_train = []
X_test = []
y_test = []
image_size = 150
for i in labels:
    folderPath = os.path.join('./Desktop/terzo anno/tesi/Brain-Tumor-Classification-DataSet-master','Training',i)
    for j in tqdm(os.listdir(folderPath)):
        img = cv2.imread(os.path.join(folderPath,j))
        img = cv2.resize(img,(image_size, image_size))
        X_train.append(img)
        y_train.append(i)
#opencv permette di di leggere l'immagine e restituisce un numpy array
for i in labels:
    folderPath = os.path.join('./Desktop/terzo anno/tesi/Brain-Tumor-Classification-DataSet-master','Testing',i)
    for j in tqdm(os.listdir(folderPath)):
        img = cv2.imread(os.path.join(folderPath,j))
        img = cv2.resize(img,(image_size,image_size))
        X_train.append(img)
        y_train.append(i)
#si convertono in numpy array
X_train = np.array(X_train)
y_train = np.array(y_train)


100%|█████████████████████████████████████████████████████████████████████████████████| 74/74 [00:00<00:00, 164.94it/s]


In [12]:
X_train,X_test, y_train, y_test = train_test_split(X_train, y_train,train_size=0.9,shuffle = False)
#shuffle uguale a false peggiora molto la rete

In [13]:
#faccio un rescaling
X_train_scaled = X_train / 255
X_test_scaled = X_test / 255

In [14]:
y_train_new = []
for i in y_train:
    y_train_new.append(labels.index(i))
y_train = y_train_new
y_train = tf.keras.utils.to_categorical(y_train)


y_test_new = []
for i in y_test:
    y_test_new.append(labels.index(i))
y_test = y_test_new
y_test = tf.keras.utils.to_categorical(y_test)

In [15]:
checkpoint = ModelCheckpoint("brain_tumor_cnn4_sgd.h5",monitor="val_accuracy",save_best_only=True,mode="auto",verbose=1)
reduce_lr = ReduceLROnPlateau(monitor = 'val_accuracy', factor = 0.3, patience = 2, min_delta = 0.001,
                              mode='auto',verbose=1)

In [18]:
cnn4.fit(X_train_scaled, y_train, epochs = 30, validation_split = 0.1, batch_size=32,callbacks=[ checkpoint, reduce_lr])

Epoch 1/30
83/83 [==============================] - ETA: 0s - loss: 1.2508 - accuracy: 0.5683
Epoch 00001: val_accuracy improved from -inf to 0.22789, saving model to brain_tumor_cnn4_sgd.h5
83/83 [==============================] - 224s 3s/step - loss: 1.2508 - accuracy: 0.5683 - val_loss: 1.6081 - val_accuracy: 0.2279
Epoch 2/30
83/83 [==============================] - ETA: 0s - loss: 0.8363 - accuracy: 0.6969
Epoch 00002: val_accuracy improved from 0.22789 to 0.79592, saving model to brain_tumor_cnn4_sgd.h5
83/83 [==============================] - 208s 3s/step - loss: 0.8363 - accuracy: 0.6969 - val_loss: 0.6029 - val_accuracy: 0.7959
Epoch 3/30
83/83 [==============================] - ETA: 0s - loss: 0.6240 - accuracy: 0.7669
Epoch 00003: val_accuracy did not improve from 0.79592
83/83 [==============================] - 211s 3s/step - loss: 0.6240 - accuracy: 0.7669 - val_loss: 1.0085 - val_accuracy: 0.7755
Epoch 4/30
83/83 [==============================] - ETA: 0s - loss: 0.5563 -

Epoch 27/30
83/83 [==============================] - ETA: 0s - loss: 0.3244 - accuracy: 0.8880
Epoch 00027: val_accuracy did not improve from 0.79592
83/83 [==============================] - 178s 2s/step - loss: 0.3244 - accuracy: 0.8880 - val_loss: 2.0525 - val_accuracy: 0.7891
Epoch 28/30
83/83 [==============================] - ETA: 0s - loss: 0.3235 - accuracy: 0.8835
Epoch 00028: val_accuracy did not improve from 0.79592

Epoch 00028: ReduceLROnPlateau reducing learning rate to 1.5943228071080283e-09.
83/83 [==============================] - 177s 2s/step - loss: 0.3235 - accuracy: 0.8835 - val_loss: 2.0548 - val_accuracy: 0.7891
Epoch 29/30
83/83 [==============================] - ETA: 0s - loss: 0.3283 - accuracy: 0.8778
Epoch 00029: val_accuracy did not improve from 0.79592
83/83 [==============================] - 178s 2s/step - loss: 0.3283 - accuracy: 0.8778 - val_loss: 2.0636 - val_accuracy: 0.7891
Epoch 30/30
83/83 [==============================] - ETA: 0s - loss: 0.3500 - 

In [19]:
checkpoint2 = ModelCheckpoint("brain_tumor_cnn4_adam.h5",monitor="val_accuracy",save_best_only=True,mode="auto",verbose=1)
reduce_lr = ReduceLROnPlateau(monitor = 'val_accuracy', factor = 0.3, patience = 2, min_delta = 0.001,
                              mode='auto',verbose=1)

In [20]:
cnn4.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adam(),
              metrics=['accuracy'])

In [21]:
cnn4.fit(X_train_scaled, y_train, epochs = 30, validation_split = 0.1, batch_size=32,callbacks=[ checkpoint2, reduce_lr])

Epoch 1/30
83/83 [==============================] - ETA: 0s - loss: 0.8869 - accuracy: 0.6693
Epoch 00001: val_accuracy improved from -inf to 0.70408, saving model to brain_tumor_cnn4_adam.h5
83/83 [==============================] - 189s 2s/step - loss: 0.8869 - accuracy: 0.6693 - val_loss: 2.3169 - val_accuracy: 0.7041
Epoch 2/30
83/83 [==============================] - ETA: 0s - loss: 0.5568 - accuracy: 0.7877
Epoch 00002: val_accuracy improved from 0.70408 to 0.79252, saving model to brain_tumor_cnn4_adam.h5
83/83 [==============================] - 189s 2s/step - loss: 0.5568 - accuracy: 0.7877 - val_loss: 1.0839 - val_accuracy: 0.7925
Epoch 3/30
83/83 [==============================] - ETA: 0s - loss: 0.4076 - accuracy: 0.8430
Epoch 00003: val_accuracy did not improve from 0.79252
83/83 [==============================] - 189s 2s/step - loss: 0.4076 - accuracy: 0.8430 - val_loss: 8.6689 - val_accuracy: 0.4320
Epoch 4/30
83/83 [==============================] - ETA: 0s - loss: 0.3673

Epoch 27/30
83/83 [==============================] - ETA: 0s - loss: 0.0612 - accuracy: 0.9830
Epoch 00027: val_accuracy did not improve from 0.81293
83/83 [==============================] - 198s 2s/step - loss: 0.0612 - accuracy: 0.9830 - val_loss: 1.3374 - val_accuracy: 0.7619
Epoch 28/30
83/83 [==============================] - ETA: 0s - loss: 0.0547 - accuracy: 0.9868
Epoch 00028: val_accuracy did not improve from 0.81293

Epoch 00028: ReduceLROnPlateau reducing learning rate to 5.314410245205181e-10.
83/83 [==============================] - 195s 2s/step - loss: 0.0547 - accuracy: 0.9868 - val_loss: 1.3335 - val_accuracy: 0.7619
Epoch 29/30
83/83 [==============================] - ETA: 0s - loss: 0.0570 - accuracy: 0.9841
Epoch 00029: val_accuracy did not improve from 0.81293
83/83 [==============================] - 203s 2s/step - loss: 0.0570 - accuracy: 0.9841 - val_loss: 1.3361 - val_accuracy: 0.7619
Epoch 30/30
83/83 [==============================] - ETA: 0s - loss: 0.0587 - a